# Machine Learning Engineer Nanodegree
## Capstone Project
Andy Wilson  
March 25, 2017

## I. Definition

### Project Overview
This project aims to construct a predictive financial model, capable of estimating the future price movement of traded financial instruments based primarily on historical price data.

This is a challenging project, since there are a range of views held as to whether this is feasible. On one hand, the Efficient Market Hypothesis (EMH) [1] argues that for efficient markets, it should not be possible to predict future prices, based on historic price information alone. On the other hand, investing strategies which rely on exactly this approach, such as momentum and trend-following approaches [2] do so with empirical success under certain conditions, suggesting that there may indeed be structure in historic price information which can be used to predict future price movements.

There are many factors that influence the price of a financial product traded on any particular market at a given time. These range from the sentiment and demand for the product by the market participants/investors to the dynamics of the systems used to operate the marketplace itself.

The hypothesis proposed here is that some of these factors are not completely independent of recent changes in the price of the product, and that by analysing enough data, some structure may be found in historic price movements that would be a predictor of future price movements.

It is appreciated that any such structure discoverable is likely to be a very weak underlying signal amid a lot of noise. Therefore it's entirely feasible, indeed likely, that any predictive signal obtained, would not be strong enough to trade profitably on it's own. Trading any systematic trading strategy incurs execution costs, typically due to price spreads (the difference between the cost to buy and that to sell a given product) and fees or commissions charged by the various brokers or marketplaces involved.

Nevertheless, it is proposed that even the identification of a weak but measurable signal would be a valuable contribution, since it could be incorporated as an additional input into existing systematic investment strategies to increase their profitability or diversity. 

#### Dataset and inputs

To have confidence in any possible discovery of a weak signal a lot of data is required. Hence the direction of the project was driven to some extent by the availability of such data. Whilst there is a wealth of financial data potentially available from market data vendors such as Bloomberg or Reuters, most of this comes at significant cost.

However, a source of freely available Foreign Exchange (FX) and Precious Metals (PM) price data was found at [4], which provides historical open/high/low/close prices sampled at 1 minute intervals across many currency pairs. In most cases, this data extends back to 2009, giving around 2.5 million samples per current pair.

This led to the proposal to attempt the prediction of short term price movements of FX currency pairs, based on recent price history.

An example extract illustrating the nature of the data available is shown below, in this case for the USDJPY FX currency pair. This shows examples of 1 minute samples of "Open/High/Low/Close" ("OHLC") price samples from one day in 2009. 

<p style="text-align: center;">USDJPY</p>

|     timestamp       |   open |   high |   low |   close |
|:--------------------|-------:|-------:|------:|--------:|
| 2009-03-16 00:00:00 |  98.25 |  98.25 | 98.23 |   98.24 |
| 2009-03-16 00:01:00 |  98.23 |  98.24 | 98.23 |   98.24 |
| 2009-03-16 00:03:00 |  98.25 |  98.25 | 98.24 |   98.24 |
| 2009-03-16 00:04:00 |  98.25 |  98.25 | 98.23 |   98.24 |
| 2009-03-16 00:05:00 |  98.23 |  98.23 | 98.22 |   98.23 |

![](reportresources/usdjpy-20090316.png)


### Problem Statement

The task proposed is to construct a predictive financial model, capable of predicting the direction of short-term future price movements, based on historical price data.

It takes as it's main input a dataset consisting of recent historical prices of a financial product and outputs a signal indicating the direction any expected future price movement within a short time period.

The input to the model for any given time point will be a snapshot of recent price information up to that time, looking back over a given time window of _W_ samples.

The model is implemented as a classifier, returning a categorical output, predicting whether the price will rise or fall. One trade-off which is investigated is that between the confidence achievable in very short term prediction with that in a longer term prediction, which may be of lower confidence but of greater price movement.

Some focus is also given to how a model trained on a single instrument's historic data compares with one trained on data from a range of instruments.

Finally, the output of the project is a trained model along with test results, analysis and measurements of it's performance on a hold-out test data set unseen during its training.

Care is be taken to avoid "data leakage" [3] (whereby implicit information from a test or evaluation scenario is inadvertently used in the training process), by ensuring that the test data sets used cover time ranges that are later in time and distinct from and data used for training.

#### Proposed Solution

The approach taken is to use a deep neural network trained on historical price information to produce a model providing predictions on future pricing movements.

Deep neural networks were chosen as they are known to be cabable of capturing arbitrarily complex patterns in high dimensional data, and they may be trained using Stochastic Gradient Descent (and variants thereof) which is a scalable approach to dealing with large volumes of data.

Note that although the latest price sample at any point in time has only a few dimensions (e.g.  open/high/low/close), the recent _history_ up until that point is a sequence of, say, _W_ elements, where _W_ is a parameter describing the length of the price history to be considered.

There are several approaches to processing sequential information for machine learning models. One approach is to use a Recurrent Neural Network (RNN), which explicitly models the input as a sequence of low-dimensional samples. Alternatively, the "sliding window" approach [6] uses one or more input features for each element of the recent history, i.e. having at least _W_ features or dimensions. This approach allows the use of regular fully-connected feed forward networks (Multi-Layer Perceptron networks).

Although both approaches are potentially interesting and worthy of comparison, in order to limit the scope, this project focusses on the use of Multi-Layer-Perceptron networks trained using the sliding window method. Some focus is given to assessing the impact of different aspects of this model architecture, such as numbers of network layers and layer sizes.


#### Benchmark Models

A number of baseline models are proposed for comparison and benchmarking as predictors of future price movements.

1. last price change predictor - a model which simply predicts the future price change to be the same as the most recent price change. Such a model may be expected to have some success at modelling short term trends.

2. moving average price change predictor - a model which predicts the future price change based on the gradient of the moving average price over a recent window. This model can itself be fine-tuned on the training dataset by selecting the optimal window size, and moving average type (e.g. simple moving average versus exponential moving average).


#### Project Design

The project followed the following workflow:

1. Download datasets for range of currency pairs and dates. 
1. Prepare training, validation and test data sets, for a single FX pair.
1. Compute descriptive statistics and visualisations of the dataset.
1. Build data pre-processing pipeline with an initial feature set (based on return series)
1. Visually investigate existence of structure in patterns of future price movement in the data, for example using T-SNE visualisations.
1. Construct baseline models, and fine tune baseline model 2 for optimal performance on training dataset.
1. Evaluate baseline models on validation data set.
1. Construct multi-layer-perceptron model
1. Train the model on the training set and evaluate the performance on the validation set.
1. Repeat with different model architectural hyperparameters (number of layers, layer width), 
1. Assess performance relative to baseline models. 
1. Investigate impact of additional features and length of lookback window
1. Investigate performance at longer term versus shorter term prediction, and select a target timescale.
1. Investigate performance when training model on dataset based on prices from multiple FX pairs.
1. Further refine hyperparameters of best performing model/dataset combination, and select this as the final model.
1. Report performance of final model on test datasets for different FX pairs, and compare with baseline model performance on the same.
1. Summarize areas for further research.



### Metrics

#### F1-Score
For the binary categorical model, which attempts to predict price rise versus price fall the F1-score metric is used for evaluation. 

Although accuracy would seem intuitively to be a reasonable metric, it is best suited for assessing performance on perfectly balanced classes. In this case, our test data are real price series which have slightly imbalanced number of positive and negative return examples. Although only a slight imbalance, this can be significant given that we are measuring performance of a relatively weak signal.

Hence F1-score which measures the balance between precision and recall and is intended for measuring performance on examples with imbalanced classes, is proposed as a more robust metric.

#### Mean Future Return

The "Mean Future Return" is also offered used as an additional metric, which we devised to provide an indicator of the model's performance from a financial perspective. It is conceived as the mean return which would be achieved over each future time period where we make a prediction, if a "cost-free" investment could be made to take advantage of the price change predicted by the model. By this we mean an investment which provides exposure to the price changes of the underlying asset, but which avoids the costs of entering and exiting positions and all trading costs. It is therefore an unrealistic measure, however it is one which introduces a financial perspective to the evaluation of the model and one which places a theoretical upper-bound on the model's financial performance.

This is computed as follows:
- for each prediction; take the actual future price return, and multiply by the sign of the model's predicted direction signal (+1 for predicted price rise, -1 for predicted price fall).
- take the mean of the above across all predictions.

First we define the return at time t as the relative price change since the last period:

$$   return_{t} = \frac{price_{t} - price_{t-1}}{price_{t-1} }$$
$$   =  price_{t} / price_{t-1} - 1 $$

Then the mean future return over _n_ future time periods:
$$meanFutureReturn =   1/n  * \Sigma_{ all t} { (return_{t+1} * sign(predictedReturn) )}  $$


## II. Analysis


### Data Exploration

In this section, you will be expected to analyze the data you are using for the problem. This data can either be in the form of a dataset (or datasets), input data (or input files), or even an environment. The type of data should be thoroughly described and, if possible, have basic statistics and information presented (such as discussion of input features or defining characteristics about the input or environment). Any abnormalities or interesting qualities about the data that may need to be addressed have been identified (such as features that need to be transformed or the possibility of outliers). Questions to ask yourself when writing this section:
- _If a dataset is present for this problem, have you thoroughly discussed certain features about the dataset? Has a data sample been provided to the reader?_
- _If a dataset is present for this problem, are statistics about the dataset calculated and reported? Have any relevant results from this calculation been discussed?_
- _If a dataset is **not** present for this problem, has discussion been made about the input space or input data for your problem?_
- _Are there any abnormalities or characteristics about the input space or dataset that need to be addressed? (categorical variables, missing values, outliers, etc.)_
-------


#### Dataset Characteristics

The FX price data available is provided as a set of "OHLC" samples taken at 1 minute intervals.
"OHLC" data contains columns representing, respectively:
- Open - the price of the first trade in the time window
- High - the highest priced trade in the time window
- Low - the lowest priced trade in the time window
- Close - the price of the last trade in the time window
where the window in question is the one minute interval. 
Each row also contains a timestamp, which by convention is the time of the start of the window.
** START / END ***???

Example extracts of the data for several currency and precious metal pairs are shown below, along with a number of charts illustrating important characteristics of the data, discussed below.

<p style="text-align: center;">EURUSD</p>

|        timestamp    |   open |   high |    low |   close |
|:--------------------|-------:|-------:|-------:|--------:|
| 2009-03-16 00:00:00 | 1.2895 | 1.2895 | 1.2895 |  1.2895 |
| 2009-03-16 00:01:00 | 1.2896 | 1.2897 | 1.2895 |  1.2895 |
| 2009-03-16 00:02:00 | 1.2896 | 1.2896 | 1.2895 |  1.2896 |
| 2009-03-16 00:03:00 | 1.2895 | 1.2896 | 1.2894 |  1.2895 |
| 2009-03-16 00:04:00 | 1.2895 | 1.2895 | 1.2894 |  1.2895 |

<p style="text-align: center;">USDJPY</p>

|        timestamp    |   open |   high |   low |   close |
|:--------------------|-------:|-------:|------:|--------:|
| 2009-03-16 00:00:00 |  98.25 |  98.25 | 98.23 |   98.24 |
| 2009-03-16 00:01:00 |  98.23 |  98.24 | 98.23 |   98.24 |
| 2009-03-16 00:03:00 |  98.25 |  98.25 | 98.24 |   98.24 |
| 2009-03-16 00:04:00 |  98.25 |  98.25 | 98.23 |   98.24 |
| 2009-03-16 00:05:00 |  98.23 |  98.23 | 98.22 |   98.23 |

<p style="text-align: center;">XAUUSD</p>

|        timestamp    |   open |   high |    low |   close |
|:--------------------|-------:|-------:|-------:|--------:|
| 2009-03-16 00:00:00 | 926.7  | 926.7  | 926.38 |  926.55 |
| 2009-03-16 00:01:00 | 926.6  | 926.6  | 926.33 |  926.55 |
| 2009-03-16 00:02:00 | 926.6  | 926.7  | 926.53 |  926.7  |
| 2009-03-16 00:03:00 | 926.65 | 926.7  | 926.65 |  926.68 |
| 2009-03-16 00:04:00 | 926.63 | 926.63 | 926.43 |  926.43 |

![](reportresources/usdjpy-feb2009-closing.png)
![](reportresources/usdjpy-1-hour.png)

This illustrates important characteristics of the data, as follows:
- some individual samples are missing. e.g. the 2009-03-16 00:02:00 sample for USDJPY. This may be because possibly no trades occurring during that window, or simply due to issues in the data collection prodess. 
- there are sometimes regular gaps in the price series. In the USDJPY February 2009 plot, these are clearly seen. In most cases these correspond to weekends or other dates when markets in these instruments are typically closed. 
- there is an order of magnitude difference in absolute price value between currency pairs.
- there is a significant long term upward trend in the price series shown in the February 2009 plot.
- the 1-hour extract illustrates the characteristics of OHLC data. Of note is that the 4 traces are of course highly correlated.

The approaches chosen for dealing with each of these characteristics are discussed below:


#### Missing samples and regular gaps due to market trading hours

Our approach to handling missing price data, for short gaps (up to 10 minutes) is to fill them with the last observed prices. Note that since we have open/high/low/close components to the price data, we fill all 4 of these features with last observed closing price from the previous period. This preserves the semantics of the OHLC features.

The process for performing this is as follows:
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
- first, we detect missing samples by resampling the data (using the pandas library 'resample' function) at 1 minute intervals. This has the effect of filling the dataset with empty records where missing samples has previously existed.
To handle occasional short gaps of missing data, up to l0 minutes in length, we simply forward fill the data. That is, the 
As we have no shortage of input data, we take a pragmatic approach to dealing with missing samples, and simply discard input records which would rely on such missing samples.

For example if we were to use a lookback window considering the previous 10 prices, then in the event of a missing sample, we would be unable to create a prediction for the next 10 time periods. Similarly, after a gap in data due to weekends of other trading holidays, we wait until we have a full window of prices before attempting to make a prediction.

Given the relatively high frequency and quantity of price samples, this works reasonably well in practice.

** REVIEW THIS - may need to resample..... <<<<<< 



#### Long term price movements

A model such as this which is trying to predict short term direction of future returns is likely to capture significant bias if trained on data where there was a long term price trend, resulting it seeing different numbers of positive versus negative examples. To counter this, we take care to normalise the input data to remove long term trends in price changes. The details of the mechanisms used are discussed below in the section on data preprocessing.


#### Different magnitude price scales

As we aim to develop a model which can be applied to any FX currency pair, the model should not be unduly affected by the magnitude of it's input data.  Furthermore it would be interesting to use historic data from more than one pair in the training of a model, in the hope that some learned features of the market behaviour would be common to different markets.

Additionally, many types of machine learning algorithm are sensitive to the scale of their input data. Hence all input data is be normalised so that the model inputs will be of similar scale. 

So rather than use the raw prices themselves, which may be arbitrarily large, and differ greatly between different FX currency pairs, the data is transformed and normalised. One important transformation is to base the features on the "returns" of the price series, meaning the relative price change from the previous time interval, which as a relative measure tend to be more comparable between different FX pairs. The "Preprocessing" section below gives full details of the transformations applied to the data, to make it more suitable for input to the models developed.

#### Correlation of OHLC features

As seen in the 1-hour chart the Open/High/Low/Close features of the data are seen to move largely in concert. Hence just one of these features should be sufficient to capture the short term historic trends in price movement across the samples. In our data preprocessing we use the returns of the closing price feature to capture this effect.

The additional features do however provide potentially useful additional information such as the range of prices within each time period and whether prices were ultimately rising or falling during each period.

For the other features, we take their values relative to the closing price to capture this information. This decorrelates them from the closing price changes (reducing redundant information), whilst keeping intact the key information that they provide, and also normalising them to similar scales.

#### Return series based on closing price

<p style="text-align: center;">EURUSD</p>

|       timestamp     |   close |       return |
|:--------------------|--------:|-------------:|
| 2009-03-16 00:00:00 |  1.2895 |  0           |
| 2009-03-16 00:01:00 |  1.2895 |  0           |
| 2009-03-16 00:02:00 |  1.2896 |  7.75494e-05 |
| 2009-03-16 00:03:00 |  1.2895 | -7.75434e-05 |
| 2009-03-16 00:04:00 |  1.2895 |  0           |
| 2009-03-16 00:06:00 |  1.2894 | -7.75494e-05 |
| 2009-03-16 00:07:00 |  1.2893 | -7.75555e-05 |
| 2009-03-16 00:08:00 |  1.2891 | -0.000155123 |

<p style="text-align: center;">USDJPY</p>

|       timestamp     |   close |       return |
|:--------------------|--------:|-------------:|
| 2009-03-16 00:00:00 |   98.24 | -0.000203542 |
| 2009-03-16 00:01:00 |   98.24 |  0           |
| 2009-03-16 00:03:00 |   98.24 |  0           |
| 2009-03-16 00:04:00 |   98.24 |  0           |
| 2009-03-16 00:05:00 |   98.23 | -0.000101792 |
| 2009-03-16 00:06:00 |   98.22 | -0.000101802 |
| 2009-03-16 00:07:00 |   98.21 | -0.000101812 |
| 2009-03-16 00:08:00 |   98.22 |  0.000101823 |

<p style="text-align: center;">XAUUSD</p>

|       timestamp     |   close |       return |
|:--------------------|--------:|-------------:|
| 2009-03-16 00:00:00 |  926.55 | -0.000194231 |
| 2009-03-16 00:01:00 |  926.55 |  0           |
| 2009-03-16 00:02:00 |  926.7  |  0.000161891 |
| 2009-03-16 00:03:00 |  926.68 | -2.1582e-05  |
| 2009-03-16 00:04:00 |  926.43 | -0.00026978  |
| 2009-03-16 00:05:00 |  926.48 |  5.39706e-05 |
| 2009-03-16 00:07:00 |  926.45 | -3.23806e-05 |
| 2009-03-16 00:08:00 |  926.4  | -5.39695e-05 |

#### Summary Statistics of Returns

|       |           EURUSD |           USDJPY |           XAUUSD |
|:------|-----------------:|-----------------:|-----------------:|
| count | 355879           | 343455           | 276108           |
| mean  |      8.96979e-08 |      1.05664e-07 |      6.56347e-07 |
| std   |      0.000229856 |      0.000254723 |      0.000341481 |
| min   |     -0.00652985  |     -0.0139469   |     -0.0078104   |
| 25%   |     -7.79636e-05 |     -0.000109637 |     -0.000126912 |
| 50%   |      0           |      0           |      0           |
| 75%   |      7.83638e-05 |      0.000109721 |      0.000129271 |
| max   |      0.0175358   |      0.00661001  |      0.00942465  |


The tables above illustrate return series produced from closing prices of 3 different FX pairs. The summary statistics and the histogram below highlight some important features of the return series:

- all have mean of approximately zero
- standard deviation of returns differ, but are of the same order of magnitude
- they have an approximately symmetrical and approximately Normal distribution.

### Exploratory Visualization

![](reportresources/histo_of_closing_price-returns.png)

The charts below compare the raw closing price of 3 pairs, with the cumulative product of the return series.
This clearly illustrates that the historical trends and features of the data are preserved by the conversion to a return series, yet the scale of the values involved are converted to similar ranges.

![](reportresources/3-pairs-closing-price-subplots.png)

![](reportresources/3-pairs-cumprod-returns.png)

?? possible add TSNE visualisation ?? or after preprocessed data introduced?

### Algorithms and Techniques
In this section, you will need to discuss the algorithms and techniques you intend to use for solving the problem. You should justify the use of each one based on the characteristics of the problem and the problem domain. Questions to ask yourself when writing this section:
- _Are the algorithms you will use, including any default variables/parameters in the project clearly defined?_
- _Are the techniques to be used thoroughly discussed and justified?_
- _Is it made clear how the input data or datasets will be handled by the algorithms and techniques chosen?_

-----
#### 
- features:
    - n-period close-to-close returns relative to latest sample, for differnt values of N
    - OHL relative to close, for each period.
    ?? check: do we do relative to that period's close, or relative to latest close. former intuitively better.
- window method
- MLP
- random forest?
- naive bayes

- SGD
- train /validation /test split / 
- approach to avoid data leakage.
    - distinct train / validation / test sets. test set is temporally after and non-overlapping with train/validation set.
    - some overlap in validation data used for learning curves.
    

#### Check for data leakage using random price series.

It is very easy, through careless data processing, to produce a solution which inadvertently leaks information from the future into processing of historic records. To guard against such accidental "data leakage", it was considered important to have some means of verifying that this had not occurred to any significant degree.
To this end, a synthetic price series was generated using a completely random series of returns. Since future price movement of such a series, is, by design, impossible to predict, the model's predictions when trained on this series, should be no better than random guessing. If the model were trained on such a series and output a prediction significantly better than random, it would be evidence to suggest that an implementation error leading to data leakage had occurred.
 
 
 
 - additional features
     - time of day
     - EWMA - close


### Benchmark
In this section, you will need to provide a clearly defined benchmark result or threshold for comparing across performances obtained by your solution. The reasoning behind the benchmark (in the case where it is not an established result) should be discussed. Questions to ask yourself when writing this section:
- _Has some result or value been provided that acts as a benchmark for measuring performance?_
- _Is it clear how this result or value was obtained (whether by data or by hypothesis)?_

------

F1 score + Mean future return for benchmark models
- score for random dataset. (0)


## III. Methodology
_(approx. 3-5 pages)_

### Data Preprocessing
In this section, all of your preprocessing steps will need to be clearly documented, if any were necessary. From the previous section, any of the abnormalities or characteristics that you identified about the dataset will be addressed and corrected here. Questions to ask yourself when writing this section:
- _If the algorithms chosen require preprocessing steps like feature selection or feature transformations, have they been properly documented?_
- _Based on the **Data Exploration** section, if there were abnormalities or characteristics that needed to be addressed, have they been properly corrected?_
- _If no preprocessing is needed, has it been made clear why?_

-----
#### Features

From the raw OHLC price data was computed additional features as follows:

For a range of periods $n\elem 0..N$:

- n-period return based on closing price
- 

##### Labels

... label generation from future returns. mean normalised, but not standard dev. scaled. (turned into binary up/down).

#### Normalisation:

##### Mean Removal
Input feature data was normalised to have approximately mean zero and standard deviation of one. The approach taken was to base this transformation on moving average values of the dataset, rather than on statistics of the batch of training data as a whole, as is often done in other problem domains.  This means that each sample is only transformed based on data from earlier in time than itself. This is important when dealing with temporal data, and avoids the subtle data leakage which occurs if say the mean of the whole batch is subtracted from each sample, whereby the resulting data would implicitly incorporate future information in their calculation.

Concretely, from each feature's current value was subtracted, the exponentially-weighted moving average (EWMA) of that feature's historic value up until that sample. The "centre-of-mass" parameter used for the EWMA calculation, (which controls the rate at which the impact of historic samples decays with their age) was chosen as 10 times the value of the largest lookback of any feature in the dataset. For example, the dataset using features based on a window of prices, looking back over 400 time periods, would use a centre-of-mass for the calculation of 400*10 = 4000 periods. This somewhat arbitrary choice was made as a trade-off between acheiving stable mean and minimising the amount of data discarded at the start of the dataset, before enough samples are available for the EWMA calculation.

The same mean-removal technique was applied both for input features, and also to the future-return data used as the basis of the label generation for training described above.

>>> INSERT EWMA calculation here <<<<

The EWMA function was chosen to determine a rolling mean value rather than a simple moving average over an equivalent window, since in general the output of an EWMA calculation tends to be smoother than that of a simple-moving average, which can experience jumps in it's output over time as outlier values fall out of the lookback window.

##### Scaling to unit Standard Deviation
Similarly, to the technique used for mean removal, to scale the samples to have a similar order of magnitude, each feature value was divided by the exponentially-weighted rolling standard-deviation of it's historic values (with the same center-of-mass parameter as used for mean removal).



**???? how have we got any data left!!!  2.8 days..

** TODO smoosh data together for normalisation, then resample and drop out NAs for features

- resample to 1 minute intervals, forward filling missing records for short intervals -> up to 10 mins
- longer intervals are allowed to generate missing records
- the moving averages used to scale mean and standard deviation, skip over missing records, and just take rolling mean of available samples.
- std of zero expliclty replaced with NA to ensure that normalisation will propagate NAs

** !! ensure 1 timestep offset in normalisation, when ewstd and ewma calculated. -> can introduce subtle data leakage.

The approach taken to normalisation was designed, so that the same approach could be used at prediction time on batches of test data, without implicitly including any future knowledge 
- mean removal. rolling EWM as smoother than rolling SMA. rolling sma can jump as samples fall out of the window.
- std scaling
- OHL relative to C
- returns for closing price
- window selection


### Implementation
In this section, the process for which metrics, algorithms, and techniques that you implemented for the given data will need to be clearly documented. It should be abundantly clear how the implementation was carried out, and discussion should be made regarding any complications that occurred during this process. Questions to ask yourself when writing this section:
- _Is it made clear how the algorithms and techniques were implemented with the given datasets or input data?_
- _Were there any complications with the original metrics or techniques that required changing prior to acquiring a solution?_
- _Was there any part of the coding process (e.g., writing complicated functions) that should be documented?_

------

- approach - analysis and experimentation done in python using jupyter notedbooks

- stable code factored out into callable python modules . available on github. [#REF]
- data loading & dataset preparation
    - parse CSV
    - example raw CSVs in github repository. more used and downloaded but not added to version control
    - module used to build dataset as pandas dataframe
    - saved as hdf5.
    - function to load
    
- stored as hdf5 for reuse across iterations of model development

- train test split

- models
    - developed using Keras framework in python. Keras supports plugable backends (Theano, Tensorflow)
    - model training done on NVidia GTX580, on Theano backend, Ubuntu 16.04 workstation.
    - trained model state saved to HDF5.
    - analysis and inference, then possible on same hardware, or Macbook, using Tensorflow backend.

- model architecture
    - input layer, dimensions to match features of dataset.
    - subsequent layers parameterised. 
    - initial 3 layer, Fx100x100 tried. subsequently refined by analysing validation set results.
    - following current recommendations #REF DL Book, use ReLU nodes
    - use of Dropout for regularisation. also parameterised and refined on validation set.
    - output layer is single sigmoid neuron

- training done using variant of stochastic gradient descent.
    - adam optimiser
    - categorical cross entropy loss function.  to match sigmoid output node. following #REF DL book
    
- training regime
    - initial investigation- fixed 100 epochs of 1 year training data.
    - training curves (training and validation error) plotted. 
    - learning rates kept at default for keras implementation.
    - verify when learning whether validation loss increasing. stop when appears to have plateaued.
    
  
- model performance evaluation.
    - metrics computed by module
    - f1-score, mean future return
    - confusion matrix to qualitatively assess predictions. e.g. are they balanced across categories.
   


### Refinement
In this section, you will need to discuss the process of improvement you made upon the algorithms and techniques you used in your implementation. For example, adjusting parameters for certain models to acquire improved solutions would fall under the refinement category. Your initial and final solutions should be reported, as well as any significant intermediate results as necessary. Questions to ask yourself when writing this section:
- _Has an initial solution been found and clearly reported?_
- _Is the process of improvement clearly documented, such as what techniques were used?_
- _Are intermediate and final solutions clearly reported as the process is improved?_

-----

- impact of more data, different features in dataset: 1yr data, versus many.
- dataset with many currency pairs contributing.
- data augmentation
- different num layers and layer width
- window size:
    - intuitively a long history can capture short and longer term trends.
    - no initial preconceptions as to what length is appropriate
    - tried last X(?) time periods initially
    - intuitively, probably need less granular time intervals as overall timespan increased. i.e if use 100 period lookback, the probably little extra value added by 101, 102 period lookback. may be 100, 120, 150 etc. i.e. use the additional features to span a greater range. Greater data efficiency.
      - approach:
          first n periods, then each i**2 for i in 1-20. 
          
           - seems to work improved performance for reduced training time.
         
- automated early stopping
    - proved tricky to tune. incorrect sensitivity would stop too early. abandoned in favour of qualitatively reviewing learning curves.
    
     

## IV. Results
_(approx. 2-3 pages)_

### Model Evaluation and Validation
In this section, the final model and any supporting qualities should be evaluated in detail. It should be clear how the final model was derived and why this model was chosen. In addition, some type of analysis should be used to validate the robustness of this model and its solution, such as manipulating the input data or environment to see how the model’s solution is affected (this is called sensitivity analysis). Questions to ask yourself when writing this section:
- _Is the final model reasonable and aligning with solution expectations? Are the final parameters of the model appropriate?_
- _Has the final model been tested with various inputs to evaluate whether the model generalizes well to unseen data?_
- _Is the model robust enough for the problem? Do small perturbations (changes) in training data or the input space greatly affect the results?_
- _Can results found from the model be trusted?_


------------

Refinement done on validation set, of data from years n.
Test data from years .... not used in model development, and used to report final results.
Model training repeated for different currency pairs, to verify robustness of approach.

### Justification
In this section, your model’s final solution and its results should be compared to the benchmark you established earlier in the project using some type of statistical analysis. You should also justify whether these results and the solution are significant enough to have solved the problem posed in the project. Questions to ask yourself when writing this section:
- _Are the final results found stronger than the benchmark result reported earlier?_
- _Have you thoroughly analyzed and discussed the final solution?_
- _Is the final solution significant enough to have solved the problem?_


--------------

## V. Conclusion
_(approx. 1-2 pages)_

### Free-Form Visualization
In this section, you will need to provide some form of visualization that emphasizes an important quality about the project. It is much more free-form, but should reasonably support a significant result or characteristic about the problem that you want to discuss. Questions to ask yourself when writing this section:
- _Have you visualized a relevant or important quality about the problem, dataset, input data, or results?_
- _Is the visualization thoroughly analyzed and discussed?_
- _If a plot is provided, are the axes, title, and datum clearly defined?_


-------------

### Reflection
In this section, you will summarize the entire end-to-end problem solution and discuss one or two particular aspects of the project you found interesting or difficult. You are expected to reflect on the project as a whole to show that you have a firm understanding of the entire process employed in your work. Questions to ask yourself when writing this section:
- _Have you thoroughly summarized the entire process you used for this project?_
- _Were there any interesting aspects of the project?_
- _Were there any difficult aspects of the project?_
- _Does the final model and solution fit your expectations for the problem, and should it be used in a general setting to solve these types of problems?_


-------------

### Improvement
In this section, you will need to provide discussion as to how one aspect of the implementation you designed could be improved. As an example, consider ways your implementation can be made more general, and what would need to be modified. You do not need to make this improvement, but the potential solutions resulting from these changes are considered and compared/contrasted to your current solution. Questions to ask yourself when writing this section:
- _Are there further improvements that could be made on the algorithms or techniques you used in this project?_
- _Were there algorithms or techniques you researched that you did not know how to implement, but would consider using if you knew how?_
- _If you used your final solution as the new benchmark, do you think an even better solution exists?_

-----------


**Before submitting, ask yourself. . .**

- Does the project report you’ve written follow a well-organized structure similar to that of the project template?
- Is each section (particularly **Analysis** and **Methodology**) written in a clear, concise and specific fashion? Are there any ambiguous terms or phrases that need clarification?
- Would the intended audience of your project be able to understand your analysis, methods, and results?
- Have you properly proof-read your project report to assure there are minimal grammatical and spelling mistakes?
- Are all the resources used for this project correctly cited and referenced?
- Is the code that implements your solution easily readable and properly commented?
- Does the code execute without error and produce results similar to those reported?


---------------------------------------------------

refinements:

The effects of including additional derived features based on price history may also be considered. For example, signals derived from different speed moving averages, which are often used in trend following investment strategies [5]. 

The effect of other inputs, such as temporal indicators, such as hour of day, and day of week may be also explored. Certainly FX markets are known to exhibit different volatility at different times of day, as market participants in different time zones become active. Whilst not necessarily contributing to a direct prediction of _direction_ of future price moves, it is considered likely that they may have impact on prediction of the distribution of _sizes_ of any price moves.

Some pre-processing of the training data sets will be performed, for example to scale and normalise them to appropriate ranges, and to de-mean the input data to remove the effect of long-term trends. A model such as this which is trying to predict short term direction of future returns is likely to capture significant bias if trained on data where there was a long term price trend, resulting it seeing different numbers of positive versus negative examples. Subtracting the mean return over the training set from each of the training samples is one potential way work around this issue, although other approaches (to be documented in the final project report) may be considered.

----

Moving average signals as features, as per [][http://localhost:8889/notebooks/fxdata-moving-average-signals.ipynb]

## References:
[1] https://en.wikipedia.org/wiki/Efficient-market_hypothesis  
[2] https://en.wikipedia.org/wiki/Momentum_(finance)  
[3] https://www.kaggle.com/wiki/Leakage  
[4] http://www.histdata.com/  
[5] https://en.wikipedia.org/wiki/Trend_following  
[6] Machine Learning for Sequential Data: A Review - Thomas G. Dietterich - http://web.engr.oregonstate.edu/~tgd/publications/mlsd-ssspr.pdf.  
